In [ ]:
# Import python packages
from snowflake.snowpark.context import get_active_session
import pandas as pd

session = get_active_session()

In [ ]:
! git clone https://github.com/wayfair/WANDS.git # Clone the WANDS repository

In [ ]:
def get_features(product_features: str):
    features = ""
    if product_features:
        for feature in product_features.split("|"):
            pair = feature.split(":")
            if len(pair) >= 2 and pair[0] and pair[1]:
                key = pair[0].strip()
                value = pair[1].strip()
                features += f"{key}: {value} "
    return features


def wands_text(row):
    text = ""
    if row["product_name"]:
        text += f"Name: {str(row['product_name']).strip()} "
    if row["product_class"]:
        text += f"Class: {str(row['product_class']).strip()} "
    if row["product_description"]:
        text += f"Description: {str(row['product_description']).strip()} "
    if row["category hierarchy"]:
        text += f"Hierarchy: {str(row['category hierarchy']).strip()} "
    if row["features"]:
        text += row['features']
    return text


product_df = pd.read_csv("WANDS/dataset/product.csv", sep="\t")
product_df["features"] = product_df["product_features"].apply(get_features)
product_df["TEXT"] = product_df.apply(wands_text, axis=1)
upload_df = product_df.rename(
    columns={
        "product_id": "ID",
        "product_name": "NAME",
        "product_class": "CLASS",
        "rating_count": "RATING_COUNT",
        "average_rating": "RATING",
        "review_count": "REVIEW_COUNT",
    }
)
upload_df = upload_df[
    [
        "ID",
        "NAME",
        "CLASS",
        "RATING_COUNT",
        "RATING",
        "REVIEW_COUNT",
        "TEXT",
    ]
]

session.write_pandas(
    df=upload_df,
    table_name="WANDS_PRODUCT_DATASET",
    schema="DATASETS",
    database="CORTEX_SEARCH_DB",
    overwrite=True,
    auto_create_table=True,
)

session.sql("""CREATE OR REPLACE CORTEX SEARCH SERVICE CORTEX_SEARCH_DB.SERVICES.WANDS
ON TEXT
ATTRIBUTES CLASS 
WAREHOUSE = WH_TEST
TARGET_LAG = '60 minute'
AS (
    SELECT
        TEXT, ID, CLASS, RATING_COUNT, RATING, REVIEW_COUNT
    FROM CORTEX_SEARCH_DB.DATASETS.WANDS_PRODUCT_DATASET
)""").collect()